In [1]:

import pandas as pd
import numpy as np
import altair as alt
from tqdm import tqdm
import graphviz
from general_consts import *
alt.data_transformers.enable('csv')
from altair_saver import save

In [3]:
measurment_no = 9025
path = fr"C:\Users\Administrator\Repositories\GreenSecurity-FirstExperiment\Dell Inc. Latitude Latitude 7430 Windows 10\Splunk Enterprise SIEM\Power Saver Plan\One Scan\Measurement {measurment_no}"

In [4]:
def get_df(path, table_name):
    df = pd.read_csv(fr"{path}\{table_name}.csv")
    df[CPUColumns.TIME] = pd.to_datetime(df[CPUColumns.TIME], unit='s')
    return df

In [5]:
cpu_df = get_df(path, TableNames.CPU)
memory_df = get_df(path, TableNames.MEMORY)
all_processes_df = get_df(path, TableNames.ALL_PROCESSES)
disk_df = get_df(path, TableNames.DISK)
splunk_logs_df = pd.read_csv(fr"{path}\output.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Administrator\\Repositories\\GreenSecurity-FirstExperiment\\Dell Inc. Latitude Latitude 7430 Windows 10\\Splunk Enterprise SIEM\\Power Saver Plan\\One Scan\\Measurement 9025\\output.csv'

In [6]:

chart = alt.Chart(cpu_df).mark_line(interpolate='linear').encode(
    alt.Y(f"{CPUColumns.USED_PERCENT}:Q"),
    alt.X(f"{CPUColumns.TIME}:T"),
).properties(
    width=2100,
    height=300
)
# save(chart, f"{path}\graphs\cpu.pdf")
chart

c:\Users\Administrator\.conda\envs\py310\lib\site-packages\altair\utils\core.py:283: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [19]:

chart = alt.Chart(disk_df).mark_line(interpolate='linear').encode(
    alt.Y(f"{DiskIOColumns.WRITE_COUNT}:Q"),
    alt.X(f"{CPUColumns.TIME}:T"),
).properties(
    width=2100,
    height=300
)
# save(chart, f"{path}\graphs\cpu.pdf")
chart

alt.Chart(...)

In [9]:

chart = alt.Chart(all_processes_df[all_processes_df['PID']==16200]).mark_line(interpolate='linear').encode(
    alt.Y("CPU(%):Q"),
    alt.X("Time(sec):T"),
).properties(
    width=1500,
    height=300
)
# .facet(    
#         row='PNAME:N'
# )

In [10]:
chart

alt.Chart(...)

In [5]:
import ctypes

# Attach to the console of the process with the specified PID
pid = 1472 # replace with the PID of the target process
kernel32 = ctypes.WinDLL('kernel32')
kernel32.AttachConsole(pid)

# Redirect stderr to a file
filename = 'output.txt'
handle = ctypes.windll.kernel32.CreateFileA(filename, 0x40000000, 0, None, 2, 0x80, None)
ctypes.windll.kernel32.SetStdHandle(-12, handle)

# Write to stderr
import sys
sys.stderr.write('This message will be redirected to the file')

# Detach from the console
kernel32.FreeConsole()


This message will be redirected to the file

1